## CSCI 3155 : Project 2 (Lettuce With Figs)

This project extends the Lettuce language with arithmetic, let bindings, function calls and recursive functions with an extra value type: _figures_. We will augment the language with operations that can allow us to 
- Draw figures made up of rectangles, lines, triangles and circles.
- Place these figures besides each other
- Rotate these figures

Your assigment for this project will include:
- Complete the interpreter for this language based on the semantics that are provided.
- Write some programs to generate some of the example figures shown.

As previously, you are given some code that includes code we have written for Lettuce with arithmetic and Booleans from the notebooks. Your job is to write the missing code. Please read this documentation properly.

__Beta Notice:__ As always, we make up things as we go along. Please feel free to point out ambiguities/ask for clarifications on piazza.





## Running the code and submitting

Environment setup is the same as for project 1.

Submission instructions are same as for project 1. Run the command
`checkAndZipSubmission` to create a submission.zip file.

## The Language Extension

We will extend Lettuce to allow us to write programs like thus

~~~
let a = rectangle(1) in
   let b = circle(0.5) in
      a/b
~~~

`a` is assigned to a rectangle of side length `1` (it will be a square) and `b` to a circle of radius `0.5`.
Thus `a,b` are of type __Figures__

We will define the division of two figures as placing the figure `a` on top of figure `b`.

The result should place a rectangle on top of a circle like thus.

<img src="circleRect.png" width=25% height=25%></img>


We are using the `/` operator to operate both on numbers or figures. This is an example of operator overloading.

Likewise we will overload other operators as follows:


|Operator | Argument 1 Type | Argument 2 Type | Meaning |
|-----| ---- | ---- | ---- | 
| +   | Number | Number | Add two numbers |
| +   | Figure(f1) | Figure(f2) | Make a composite figure by _overlapping_ the two figures f1, f2 | 
| *   | Number | Number | Multiply two numbers | 
| *   | Figure(f1) | Figure(f2) | Make a composite figure by placing figure f2 to the right of figure f1 | 
| /   | Number | Number | Division |
| /   | Figure(f1) | Figure(f2) | Place figure f1 on top of figure f2 |
| /  | Figure(f1) | Number (d) | Rotate figure f1 around (0,0) by angle d in radians | 
| -   | Number | Number | Subtract | 

Note that all other operations are meaningless. For instance, trying to _subtract_ one figure from other
leads to error. Likewise, adding a number to a figure leads to an error.

The table above is going to be very important. Please refer back to it when you interpret the programs below.

What does the program below do?

~~~
let f1 = triangle(1) in 
  (f1 * f1)/f1
~~~

<img src="program1-result.png" width=25% height=25%></img>

The star operator places two triangles one besides other and the / operator places a new triangle on top.


The language allows functions as well:

~~~
let f = function (x) 
      rectangle(x) * ((triangle(x)/triangle(x)) * circle(x))
      in 
    f(1) + f(2) / f(3)
~~~

Can you try interpreting what figure will be generated by the program above?

<img src="program2-result.png" width=40% height=40%></img>

We will also include recursion and now interesting things happen.

~~~

letrec serp = function (x)
       if (x <= 1)
       then (triangle(x)*triangle(x))/triangle(x)
       else (
           let f = serp(x-1) in
               (f * f) / f
            ) in
       serp(5)
       
~~~

<img src="serpinski-result.png" width=40% height=40%></img>

Or try this one

~~~
letrec mandala = function (x)
    let ang = 3.1415/10 in
    if (x <= 1)
    then rectangle(2)+triangle(2)
    else (rectangle(2)+triangle(2))+(mandala(x-1)/ang)
  in
  mandala(20)
~~~

<img src="mandala-result.png" width=40% height=40%></img>

Intrigued? Interested in bringing this language to life? March on forward.


## Abstract Syntax of LettuceWithFig

We will have the following abstract syntax borrowed from the language with recursive calls. Note that the key 
extensions are shown in red.

$$\newcommand\nt[1]{\mathbf{#1}}$$

A program is simply a expression wrapped around the _TopLevel_ constructor.

$$\begin{array}{rcl}
\nt{Program} & \rightarrow & TopLevel(\nt{Expr}) \\
\end{array}$$

Expressions are given as follows:

$$\begin{array}{rcll}
\nt{Expr} & \rightarrow & Const(\nt{Double}) \\
& \rightarrow  & ConstBool(\nt{Boolean}) \\
& \rightarrow & Ident(\nt{String}) \\ \\
& \rightarrow & \color{red}{Rectangle(\nt{Expr})}  & \text{Creation of Basic Shapes for Figures} \\
& \rightarrow & \color{red}{Triangle(\nt{Expr})} \\
& \rightarrow & \color{red}{Circle(\nt{Expr})} \\
& \rightarrow & \color{red}{Line(\nt{Expr})} \\ \\
& \rightarrow & Plus(\nt{Expr}, \nt{Expr}) & \text{Arithmetic Operators are overloaded for figures as discussed previously}\\
& \rightarrow & Minus(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & Mult(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & Div(\nt{Expr}, \nt{Expr}) \\
&& \\
& \rightarrow & Geq(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & Gt(\nt{Expr}, \nt{Expr} ) \\
& \rightarrow & Eq(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & Neq(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & And(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & Or(\nt{Expr}, \nt{Expr}) \\
& \rightarrow & Not(\nt{Expr})\\ \\
    & \rightarrow & IfThenElse(\nt{Expr}, \nt{Expr}, \nt{Expr})\\\\
& \rightarrow & Let(\nt{String}, \nt{Expr}, \nt{Expr}) & \text{let bindings}\\
& \rightarrow & LetRec(\nt{String}, \nt{String}, \nt{Expr}, \nt{Expr}) & \text{recursive definition} \\
& \rightarrow & FunDef(\nt{String}, \nt{Expr}) & \text{anonymous function}\\
& \rightarrow & FunCall(\nt{Expr}, \nt{Expr}) & \text{function call}\\
\end{array}$$


## Canvas: A value type for representing figures.

We are first going to define a value type called __Canvas__. A canvas is going to collect a bunch of shapes with coordinates. We will define some basic operations on canvases:
- creating a canvas with a single shape.
- bounding box for a canvas.
- overlap of two canvases.
- placing one canvas on top of another to yield a new canvas.
- placing one canvas to the left of another to yield a new canvas.
- rotation of a canvas.


A canvas is  of the form  $Canvas(\{o_1, \ldots, o_n\})$, wherein $o_1, \ldots, o_n$ are the objects in the canvas.
Each object $o_i$ is itself one of the following: 
- $Polygon(\{ (x_0, y_0), \ldots, (x_{k-1}, y_{k-1}) \})$: A polygon whose vertex coordinates are specified.
- $Circle( (x_c, y_c), r) $: A circle with the specified center $(x_c, y_c)$ and radius.


### Example of Canvas

Consider a canvas 

~~~
Canvas ( { 
    Polygon( { (0,0), (1,0), (0.5,0.5) } ) , 
    Circle ( (1,1), 1) , 
    Polygon( { (1,1), (2,1), (2,2), (1,2) }  ) 
    } ) 
~~~


<img src="canvas-img1.png" width=50% height=50%></img>


Canvases are going to be a value type for our interpreter along with numbers, booleans, closures and (nominally) error.


## Operations on canvas.

### Bounding Box of a canvas.

A bounding box for a canvas $c$ is given by $\texttt{boundingBox}(c)$ which 
is a tuple of $4$ numbers of the form $(x_{\min}, x_{\max}, y_{\min}, y_{\max})$ where 
- $x_{min}$ is the smallest $x$ coordinate for any object in the canvas.
- $x_{max}$ is the largest $x$ coordinate for any object in the canvas.
- $y_{min}$ is the smallest $y$ coordinate for any object in the canvas.
- $y_{max}$ is the largest $y$ corrdinate for any object in the canvas.

Once again consider the example canvas from before:

~~~
Canvas ( { 
    Polygon( { (0,0), (1,0), (0.5,0.5) } ) , 
    Circle ( (1,1), 1) , 
    Polygon( { (1,1), (2,1), (2,2), (1,2) }  ) 
    } ) 
~~~

Its bounding box is given by $(0, 2, 0, 2)$ with $x_{\min}= y_{\min} = 0$ and $x_{\max} = y_{\max} = 2$.

### Translate operation on a canvas

Given a Canvas object $c$ , its translation by $(xShift, yShift)$ 
is denoted $\texttt{translate}(c, xShift, yShift)$ and yields a new canvas.

The translate operation adds the value `xShift` to all x coordinates of the
object in the canvas and `yShift` to all y coordinates of the object in the canvas.

Once again consider the example canvas from before:
~~~
Canvas ( { 
    Polygon( { (0,0), (1,0), (0.5,0.5) } ) , 
    Circle ( (1,1), 1) , 
    Polygon( { (1,1), (2,1), (2,2), (1,2) }  ) 
    } ) 
~~~

We wish to perform a translation by $(2,3)$ where xShift = 2 and yShift = 3.
The result will be the canvas.

~~~
Canvas ( { 
    Polygon( { (2,3), (3,3), (2.5,3.5) } ) , 
    Circle ( (3,4), 1) , 
    Polygon( { (3,4), (4,4), (4,5), (3,5) }  ) 
    } ) 
~~~

### Rotation Operation on a canvas.

Given a Canvas object $c$ , its rotation by $\theta$ in radians 
is denoted $\texttt{rotate}(c, \theta)$ and yields a new canvas.


Recall from your calculus class that rotating a point $(x,y)$ by angle $\theta$ in radians yields

$ x' = x \cos(\theta) - y \sin(\theta),\ \ \ y' = x\sin(\theta)  + y\cos(\theta)$

As an example, let us rotate the canvas from before:
~~~
Canvas ( { 
    Polygon( { (0,0), (1,0), (0.5,0.5) } ) , 
    Circle ( (1,1), 1) , 
    Polygon( { (1,1), (2,1), (2,2), (1,2) }  ) 
    } ) 
~~~

by $30^{\circ}$ or $\frac{\pi}{6}$ radians. 

Rotating a canvas is given by rotating each object in the canvas.
- Rotating a polygon is given by applying rotation by $\theta$ to each vertex
- Rotating a circle is given by applying rotation by $\theta$ to the center of the circle. Radius is unchanged.


### Overlapping one canvas with other.

Let $c1: Canvas(O_1)$ with objects in set $O_1$ and likewise $c_2: Canvas(O_2)$. We define
the canvas

$\texttt{overlap}(c1, c2) = Canvas(O_1 \cup O_2)$

The new canvas has objects which are simply the union of objects from canvases c1 and c2.

### Placing one canvas on top of another

The `placeTop` operator will place one canvas on top of another. Let us consider the operation of 
placing canvas `c2` on top of canvas `c1`. To achieve this, we are going to first
- translate the objects in canvas `c2` by a `shiftX` and `shiftY` that we will compute.
- next we will make a new canvas that combines the objects in both.

<img src="placeTop-illustration.png" width=50% height=50%> </img>

Let $c_1: Canvas(O_1)$ and $c_2: Canvas(O_2)$ be the two canvases. The operation 
$\texttt{placeTop}(c_1, c_2)$ is defined as follows:

- Let $(x_{\min,1}, x_{\max,1}, y_{\min,1}, y_{\max,1})$ be the result of $\texttt{boundingBox}(c_1)$.
- Let $\color{blue}{(x_{\min,2}, x_{\max,2}, y_{\min,2}, y_{\max,2})}$ be the result of $\texttt{boundingBox}(c_2)$.
- Define $xShift = (x_{\max,1} - x_{\min,1})/2 -  \color{blue}{(x_{\max,2} - x_{\min,2})}/2$.
- Define $yShift = ( y_{\max,1} - y_{\min,1})$.
- Let $\color{red}{\hat{c}_2}$ be the canvas obtained by $\texttt{transate}(c_2, xShift, yShift) $
- Define the result of $\texttt{placeTop}(c_1, c_2) = \texttt{overlap}(c_1, \color{red}{\hat{c}_2}) $.

The calculations above sound somewhat arbitrary but can be easily justified if you look at the figure above.


### Placing one canvas to the right of another

The `placeRight` operator will place one canvas on top of another. Let us consider the operation of 
placing canvas `c2` to the right of canvas `c1`. To achieve this, we are going to first
- translate the objects in canvas `c2` by a `shiftX` and `shiftY` that we will compute.
- next we will make a new canvas that combines the objects in both.

<img src="placeRight-illustration.png" width=50% height=50%></img>

Let $c_1: Canvas(O_1)$ and $c_2: Canvas(O_2)$ be the two canvases. The operation 
$\texttt{placeRight}(c_1, c_2)$ is defined as follows:

- Let $(x_{\min,1}, x_{\max,1}, y_{\min,1}, y_{\max,1})$ be the result of $\texttt{boundingBox}(c_1)$.
- Let $\color{blue}{(x_{\min,2}, x_{\max,2}, y_{\min,2}, y_{\max,2})}$ be the result of $\texttt{boundingBox}(c_2)$.
- Define $xShift = (x_{\max,1} - x_{\min,1})$.
- Define $yShift = ( y_{\max,1} - y_{\min,1})/2 - \color{blue}{(y_{\max,2} - y_{\min,2})}/2$.
- Let $\color{red}{\hat{c}_2}$ be the canvas obtained by $\texttt{transate}(c_2, xShift, yShift) $
- Define the result of $\texttt{placeRight}(c_1, c_2) = \texttt{overlap}(c_1, \color{red}{\hat{c}_2}) $.

The calculations above sound somewhat arbitrary but can be easily justified if you look at the figure above.



## Task 1

Implement the canvas operations using the data structures defined in the file `MyCanvas.scala`.
Please follow the given scheme and do not modify existing code.

Your results must pass the unit tests provided under `CanvasTests`

## Interpreting Lettuce With Figs.

We will interpret our language now. We will have the following value types:
- NumValue(Double)
- BoolValue(Boolean)
- Closure(String, Expression, Environment)
- FigValue(Canvas)
- Error 

Note that we will not really consider error and just throw an exception whenever we encounter error.

For your convenience the operations over values are defined in the file `Value.scala`. Notice some 
refactoring is happening to help out with implementing the operator.

- You may want to modify the functions
 - ValueOps.plus, ValueOps.mult, ValueOps.divide, ValueOps.minus and so on.
 
 
 The semantics of Lettuce with Figs are going to inherit all the existing semantic rules used for handling
 - arithmetic
 - booleans
 - comparisons
 - function calls
 - let bindings
 - recursion
 
 The additional rules need to deal with operations over figures. We provide these rules below.

### Semantics of Figs.

The first few rules specify the creation of basic figures.

$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\ \ (\text{#3})}$
$\newcommand\reals{\mathbb{R}}$
$\newcommand\evalExpr{\mathsf{evalExpr}}$

The rule for a rectangle is as follows:

$\semRule{ \evalExpr(e, \sigma) = v, \ v \in \reals } { \evalExpr(\texttt{Rectangle(e)}, \sigma) = Canvas(\{ Polygon( \{ (0,0), (0,v), (v,v), (v,0) \} ) \}) }{rectangle-ok}$

The rule tells us how to create a rectangle whose side length is given by the expression `e`. 
The error rules for this is simple and left to the reader.

The rule for a triangle is as follows:

$\semRule{ \evalExpr(e, \sigma) = v, \ v \in \reals } { \evalExpr(\texttt{Triangle(e)}, \sigma) = Canvas(\left\{ Polygon( \left\{ (0,0), (v,0), \left(\frac{v}{2}, \frac{\sqrt{3}v}{2}\right) \right\} ) \right\}) }{triangle-ok}$

Note that the vertex $(\frac{v}{2}, \frac{\sqrt{3}v}{2})$ ensures that the triangle is equilateral.

The rules for a circle and line are as follows:

$\semRule{ \evalExpr(e, \sigma) = v, \ v \in \reals } { \evalExpr(\texttt{Circle(e)}, \sigma) = Canvas(\{ Circle((v,v), v) \}) }{circle-ok}$

$\semRule{ \evalExpr(e, \sigma) = v, \ v \in \reals } { \evalExpr(\texttt{Line(e)}, \sigma) = Canvas(\{ Polygon (\{ (0,0), (v,0) \}) \}) }{line-ok}$


The rule for error is simple:

$\semRule{ \evalExpr(e, \sigma) = v, \ \color{red}{v \not\in \reals}, \ \texttt{T} \in \{\texttt{Line}, \texttt{Circle}, \texttt{Rectangle}, \texttt{Triangle}\} } { \evalExpr(\texttt{T(e)}, \sigma) = \mathbf{error} }{basic-shape-error}$


### Overloading Arithmetic Operators on Figures

We will talk about the semantics of arithmetic operators on figures. Here is the helpful table from before.


|Operator | Argument 1 Type | Argument 2 Type | Meaning |
|-----| ---- | ---- | ---- | 
| +   | Number | Number | Add two numbers |
| +   | Figure(f1) | Figure(f2) | Make a composite figure by _overlapping_ the two figures f1, f2 | 
| *   | Number | Number | Multiply two numbers | 
| *   | Figure(f1) | Figure(f2) | Make a composite figure by placing figure f2 to the right of figure f1 | 
| /   | Number | Number | Division |
| /   | Figure(f1) | Figure(f2) | Place figure f1 on top of figure f2 |
| /  | Figure(f1) | Number (d) | Rotate figure f1 around (0,0) by angle d in radians | 
| -   | Number | Number | Subtract | 


Rules for arithmetic opertations:

### Rule for Plus 

$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 \in \reals,\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 \in \reals,\  }{\evalExpr(\texttt{Plus(e1, e2)}, \sigma) = v_1 + v_2 }{plus-numbers-ok}$

The rule above is the usual rule for plus. We also have a rule for plus involving figures. 

$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 = Canvas(O_1)\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 = Canvas(O_2)}{\evalExpr(\texttt{Plus(e1, e2)}, \sigma) = \texttt{overlap}(v_1, v_2) }{plus-figures-ok}$

We are not writing the error rules. But note that adding any other combination of values other than a number plus number or figure plus figure results in an error.

### Rule For Multiplication


$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 \in \reals,\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 \in \reals,\  }{\evalExpr(\texttt{Mult(e1, e2)}, \sigma) = v_1 \times v_2 }{mult-numbers-ok}$

The rule above is the usual rule for multiplication of numbers. We also have a rule  involving figures. 

$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 = Canvas(O_1)\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 = Canvas(O_2)}{\evalExpr(\texttt{Plus(e1, e2)}, \sigma) = \texttt{placeRight}(v_1, v_2) }{mult-figures-ok}$

__Note__ Place the figure generated by `e2` to the right of that generated by `e1`.


Similarly we are skipping error rules that cover all other combinations (other than multiplying a number by a number or multiplying a figure by a figure)


### Rule for Division


Division of two numbers (usual rule): 

$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 \in \reals,\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 \in \reals,\  v_2 \not= 0 }{\evalExpr(\texttt{Div(e1, e2)}, \sigma) = v_1/v_2 }{div-numbers-ok}$


Division of one figure by another:

$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 = Canvas(O_1)\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 = Canvas(O_2)}{\evalExpr(\texttt{Div(e1, e2)}, \sigma) = \texttt{placeTop}(v_2, v_1) }{div-figures-ok}$

__Note__ Place the figure generated by `e1` on top of that generated by `e2`.

Division of a figure by a number: 

$\semRule{\evalExpr(\texttt{e1}, \sigma) = v_1,\ v_1 = Canvas(O_1)\  \evalExpr(\texttt{e1}, \sigma) = v_2,\ v_2 \in \reals}{\evalExpr(\texttt{Div(e1, e2)}, \sigma) = \texttt{rotate}(v_1, v_2) }{div-figure-number}$

__Note__ Rotate the figure generated by `e1` using the angle in radians that expression `e2` evaluates to.

### Rule for Other Operations

The rules for the other operations, let bindings, function definitions, function calls and recursion remain unchanged.



## Task 2: Write the Interpreter

Your job is to fill in the missing code for the interpreter. 

The interpreter is defined in the file `Interpreter.scala`. 

The value types are in `Value.scala`

The environment (for handling recursion as well) is defined in `Environment.scala`.

You can throw an `IllegalArgumentException` whenever you encounter an error. A string message should be 
added to the exception to indicate why you fail.